# Neighbourhood Clustering Toronto

## This is for Capstone Datascience Project IBM and Coursera

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as requests

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {
     'class' : 'wikitable sortable' }).tbody
rows = table.find_all('tr')
columns = [v.text.replace('\n', "") for v in rows[0].find_all('th')]
trnt = pd.DataFrame(columns = columns)
trnt


,Postal Code,Borough,Neighborhood


In [13]:
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text[:-1]
            i = i + 1
        elif i == 1:
            borough = td.text[:-1]
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    trnt = trnt.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

trnt = trnt[trnt.Borough!='Not assigned']
trnt = trnt[trnt.Borough!= 0]
trnt.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,trnt.shape[0]):
    if trnt.iloc[i][2] == 'Not assigned':
        trnt.iloc[i][2] = trnt.iloc[i][1]
        i = i+1
                                 
df = trnt.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
output = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
output


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [23]:
output.rename(columns={'Postalcode': 'Postal Code'}, inplace=True)
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
output = output.merge(geo_df, how='left')
output

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [25]:
!pip install conda

  ERROR: Could not find a version that satisfies the requirement menuinst (from conda) (from versions: none)
ERROR: No matching distribution found for menuinst (from conda)
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [29]:
!pip install sklearn

You should consider upgrading via the 'python -m pip install --upgrade pip' command.



  Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


In [30]:

import folium
import numpy as np
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

KeyboardInterrupt: 

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(output['Latitude'],output['Longitude'],output['Borough'],output['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [ ]:

k=5
toronto_clustering = output.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
output.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(output['Latitude'], output['Longitude'], output['Neighborhood'], output['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters